In [1]:
#This is a generic chat RAG with pdf document. Keep your pdf document in the documents folder
#otherwises you change the folder name and keep the pdf documents there,
#It was run on a physics tutorial

In [2]:
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader

In [3]:
loader = DirectoryLoader(
    path="I:\Sridhar\\Data\\Java",
    glob="*.pdf",
    loader_cls=PyPDFLoader
)

In [4]:
documents = loader.load()

In [5]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [6]:
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)

In [7]:
chunks = splitter.split_documents(documents)

In [8]:
print(f"Total chunks: {len(chunks)}")

Total chunks: 572


In [9]:
for chunk in chunks[1:2]:
    print(chunk)

page_content='area .
[doctor] nice . that's great . i'm glad to hear that you're staying active , you know . i , i just love this 
weather . i'm so happy the summer is over . i'm definitely more of a fall person .
[patient] yes , fall foliage is the best .
[doctor] yeah . um , so tell me , how are you doing with the congestive heart failure ? how are you 
doing watching your diet ? i know we've talked about watching a low sodium diet . are you doing 
okay with that ?
[patient] i've been doing well with that . i resisted , as much , as i could , from the tater tots , you 
know , the soft pretzels , the salty foods that i , i love to eat . and i've been doing a really good job .
[doctor] okay , all right . well , i'm glad to hear that . and you're taking your medication ?
[patient] yes .
[doctor] okay , good . and any symptoms like chest pains , shortness of breath , any swelling in 
your legs ?
[patient] no , not that i've noticed .' metadata={'producer': 'Microsoft: Print To PDF', 'cre

In [10]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

In [11]:
from langchain.vectorstores import Chroma

chroma_store = Chroma.from_documents(
    documents=chunks,
    embedding=embeddings,
    persist_directory="I:/Sridhar/Chroma_DB_PN"
)


In [12]:
retriever = chroma_store.as_retriever(search_kwargs={"k": 50})

In [13]:
#results = retriever.invoke("asprin")

In [14]:
#for result in results:
#    print(result.page_content)

In [15]:
from langchain.chat_models import init_chat_model
model = init_chat_model("gpt-4o", model_provider="openai")

In [16]:
from langchain.prompts import ChatPromptTemplate

# Define a chat prompt template with context and user question
template = ChatPromptTemplate.from_messages([
    ("system", "You are a professional summarizer tasked with creating a concise, comprehensive summary of the full document. Maintain neutrality, preserve key ideas, and write in formal, clear language."),
    ("user", "Context: {context}\n\nQuestion: {question}")
])


In [23]:
# User's question
user_question = "Please provide me a summary of Stephanine's ailments, symptoms and treatment provided"

# Retrieve relevant documents
docs = retriever.invoke(user_question)
context = "\n".join([doc.page_content for doc in docs])


In [24]:
# Format the prompt with retrieved context and the user's question
formatted_prompt = template.format(context=context, question=user_question)

In [25]:
#from langchain.llms import OpenAI  # or your preferred LLM

#llm = OpenAI(model="gpt-3.5-turbo")  # adjust model as needed

# Get the answer
response = model.invoke(formatted_prompt)
print(response.content)


Stephanie is a 49-year-old female with a medical history of congestive heart failure, kidney stones, and a prior colonoscopy, who presented with an abnormal lab finding, specifically low hemoglobin levels. Her symptoms include persistent fatigue and dizziness over the past few months, leading to exhaustion from activities such as walking a mile or working. The doctor plans to assist her in scheduling necessary follow-up appointments. Additionally, Stephanie requires a prescription for allopurinol, which the doctor ordered at 100 milligrams once daily to address her needs. Discussions regarding her care continue, with the doctor noting that the nurse will help schedule necessary procedures or follow-ups as part of her treatment plan.


In [ ]:
#https://vitaliihonchar.com/insights/python-rag-api